In [5]:
# 📌 CONFIGURACIÓN DEL PROYECTO

# Umbral NDWI para detectar agua
NDWI_THRESHOLD = 0.2

# Tamaño del buffer (en metros) para agrupar polígonos cercanos
BUFFER_SIZE = 20

# Área mínima (en m²) de los polígonos a conservar
MIN_POLYGON_AREA = 300

# Porcentaje máximo de nubes permitido
CLOUD_COVER_MAX = 10

# Colección Sentinel a usar
SENTINEL_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'

# Carpeta de exportación en Google Drive
EXPORT_FOLDER = 'Azolla Filiculoides - Geojson Lagunas'

# Activar suavizado individual de polígonos (cierra huecos internos sin unir lagos entre sí)
SMOOTH_POLYGONS = True
SMOOTH_BUFFER_SIZE = 40  # ⬅️ ayuda a cerrar huecos internos

# 📦 Imports
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from ipyleaflet import WidgetControl
from ipywidgets import HTML

# 🌍 Inicialización Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# 🗺️ Mapa base
Map = geemap.Map(center=[-36.5, -60], zoom=4)
Map.add_basemap('Esri.WorldImagery')
display(Map)

# 🎚️ NDWI Slider
ndwi_slider = widgets.FloatSlider(
    value=NDWI_THRESHOLD, min=0.1, max=0.5, step=0.01,
    description='NDWI Threshold:', continuous_update=False
)
display(ndwi_slider)

# 🧹 Función para limpiar capas previas
def remove_ee_layers(map_obj):
    ee_layers = [lyr for lyr in map_obj.layers if hasattr(lyr, 'ee_object')]
    for lyr in ee_layers:
        map_obj.remove_layer(lyr)

# 🧠 FUNCIÓN PRINCIPAL: Dividir Argentina en 6 regiones y exportar una por una
def extract_water_polygons_tiled(ndwi_threshold):
    argentina = ee.FeatureCollection("FAO/GAUL/2015/level0") \
        .filter(ee.Filter.eq('ADM0_NAME', 'Argentina')) \
        .geometry()

    print("🌎 Dividiendo Argentina en 6 regiones para evitar timeout...")

    # ---- Helper: float-based range generator
    def frange(start, stop, step):
        while start < stop:
            yield start
            start += step

    # ---- Obtener límites del país
    bounds = argentina.bounds().coordinates().get(0).getInfo()
    xmin, ymin = bounds[0][0], bounds[0][1]
    xmax, ymax = bounds[2][0], bounds[2][1]

    # ---- Crear 3x2 grilla (6 tiles)
    nx, ny = 3, 2
    dx = (xmax - xmin) / nx
    dy = (ymax - ymin) / ny

    xs = list(frange(xmin, xmax, dx))
    ys = list(frange(ymin, ymax, dy))

    tile_id = 0
    for i, x in enumerate(xs[:-1]):
        for j, y in enumerate(ys[:-1]):
            rect = ee.Geometry.Rectangle([x, y, x + dx, y + dy])
            region = rect.intersection(argentina, maxError=1000)
            if region.area().getInfo() == 0:
                continue

            tile_id += 1
            print(f"🟦 Procesando tile {tile_id} (col {i+1}, fila {j+1})...")

            # 📦 Imagen Sentinel-2
            s2 = ee.ImageCollection(SENTINEL_COLLECTION) \
                .filterBounds(region) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUD_COVER_MAX)) \
                .sort('system:time_start', False) \
                .first()

            if s2 is None:
                print(f"⚠️ No se encontró imagen válida para tile {tile_id}")
                continue

            img_date = s2.date().format('YYYY-MM-dd').getInfo()

            # 💧 NDWI
            ndwi = s2.normalizedDifference(['B3', 'B8']).rename('NDWI')
            water_mask = ndwi.gt(ndwi_threshold)
            water_mask_int = water_mask.updateMask(water_mask).multiply(1).toInt()

            # 🔹 Vectorización
            vectors = water_mask_int.reduceToVectors(
                geometry=region,
                scale=30,
                geometryType='polygon',
                eightConnected=True,
                bestEffort=True,
                maxPixels=1e13
            )

            vectors = vectors.map(lambda f: f.set('area', f.geometry().area(1)))
            vectors = vectors.filter(ee.Filter.gt('area', MIN_POLYGON_AREA))

            # 🔁 Suavizado de polígonos
            if SMOOTH_POLYGONS:
                def smooth_and_fill(f):
                    g = f.geometry().buffer(BUFFER_SIZE).buffer(-BUFFER_SIZE)
                    return ee.Feature(g).copyProperties(f)
                vectors = vectors.map(smooth_and_fill)

            export_name = f'agua_ARG_tile{tile_id}_{img_date.replace("-", "")}_ndwi{int(ndwi_threshold*100)}'
            task = ee.batch.Export.table.toDrive(
                collection=vectors,
                description=export_name,
                folder=EXPORT_FOLDER,
                fileNamePrefix=export_name,
                fileFormat='GeoJSON'
            )
            task.start()
            print(f"🚀 Tile {tile_id} exportado a Drive como {export_name}.geojson")

    print(f"✅ Proceso completo. {tile_id} tiles exportados a Drive.")

# 🎯 Ejecutar manualmente o con el slider
ndwi_slider.observe(lambda change: extract_water_polygons_tiled(ndwi_slider.value), names='value')

# Para ejecutar directamente sin usar el slider:
# extract_water_polygons_tiled(0.2)


Map(center=[-36.5, -60], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

FloatSlider(value=0.2, continuous_update=False, description='NDWI Threshold:', max=0.5, min=0.1, step=0.01)

In [8]:
extract_water_polygons_tiled(0.2)

🌎 Dividiendo Argentina en 6 regiones para evitar timeout...
🟦 Procesando tile 1 (col 1, fila 1)...


EEException: Request had insufficient authentication scopes.

In [7]:
import ee
ee.Authenticate(scopes=['https://www.googleapis.com/auth/earthengine',
                        'https://www.googleapis.com/auth/drive'])
ee.Initialize()